In [0]:
x = 10

In [0]:
x

Out[3]: 10

In [0]:
from pyspark import SparkContext

In [0]:
rdd1 = sc.parallelize([0,1,2,3,4,5])

In [0]:
type(rdd1)

Out[9]: pyspark.rdd.RDD

In [0]:
sales_rdd = sc.textFile('/FileStore/tables/5000_Sales_Records.csv')

In [0]:
type(sales_rdd)

Out[12]: pyspark.rdd.RDD

In [0]:
sales_rdd.take(5)

Out[13]: ['Region,Country,Item Type,Sales Channel,Order Priority,Order Date,Order ID,Ship Date,Units Sold,Unit Price,Unit Cost,Total Revenue,Total Cost,Total Profit',
 'Central America and the Caribbean,Antigua and Barbuda ,Baby Food,Online,M,12/20/2013,957081544,1/11/2014,552,255.28,159.42,140914.56,87999.84,52914.72',
 'Central America and the Caribbean,Panama,Snacks,Offline,C,7/5/2010,301644504,7/26/2010,2167,152.58,97.44,330640.86,211152.48,119488.38',
 'Europe,Czech Republic,Beverages,Offline,C,9/12/2011,478051030,9/29/2011,4778,47.45,31.79,226716.10,151892.62,74823.48',
 'Asia,North Korea,Cereal,Offline,L,5/13/2010,892599952,6/15/2010,9016,205.70,117.11,1854591.20,1055863.76,798727.44']

In [0]:
header = sales_rdd.first()
sales_rdd = sales_rdd.filter(lambda x : x != header)
sales_rdd = sales_rdd.map(lambda line: line.split(","))

In [0]:
# # 1. Display the number of countries present in the data(Using Pyspark)
countries = sales_rdd.map(lambda x : x[1]).distinct()

In [0]:
countries.collect()

Out[18]: ['Antigua and Barbuda ',
 'North Korea',
 'Federated States of Micronesia',
 'Ethiopia',
 'Saint Lucia',
 'Lebanon',
 'Austria',
 'Mexico',
 'Trinidad and Tobago',
 'Libya',
 'Algeria',
 'Estonia',
 'Saudi Arabia',
 'Montenegro',
 'Guatemala',
 'Australia',
 'Malawi',
 'Somalia',
 'Switzerland',
 'Laos',
 'Angola',
 'Mauritania',
 'Finland',
 'Belgium',
 'Kiribati',
 'Uzbekistan',
 'South Korea',
 'Nigeria',
 'South Africa',
 'Netherlands',
 'Solomon Islands',
 'Iran',
 'Equatorial Guinea',
 'Iraq',
 'Mauritius ',
 'Eritrea',
 'Ukraine',
 'Myanmar',
 'Latvia',
 'Portugal',
 'Barbados',
 'Poland',
 'Zambia',
 'Slovenia',
 'Bhutan',
 'Cyprus',
 'Monaco',
 'Gabon',
 'Norway',
 'Thailand',
 'Tanzania',
 'Denmark',
 'China',
 'United States of America',
 'Philippines',
 'Kuwait',
 'Turkmenistan',
 'Kosovo',
 'Hungary',
 'Pakistan',
 'Mozambique',
 'Kazakhstan',
 'East Timor',
 'Liberia',
 'Albania',
 'Moldova ',
 'India',
 'Republic of the Congo',
 'France',
 "Cote d'Ivoire",
 'Cos

In [0]:
countries.coalesce(1).saveAsTextFile('/FileStore/tables/country_orders')

In [0]:
# 2.Display the number of units sold in each region.(Using Pyspark)

rdd_region_units = sales_rdd.map(lambda x : (x[0],int(x[8])))
rdd_region_units = rdd_region_units.reduceByKey(lambda a,b : a+b)
rdd_region_units.collect()

Out[20]: [('Asia', 3620036),
 ('Middle East and North Africa', 3013431),
 ('Australia and Oceania', 2111786),
 ('Central America and the Caribbean', 2698776),
 ('Europe', 6582322),
 ('Sub-Saharan Africa', 6642380),
 ('North America', 484760)]

In [0]:
rdd_region_units.coalesce(1).saveAsTextFile('/FileStore/tables/region_wise_count_units')

In [0]:
# 3.Display the 10 most recent sales. (Using Pyspark) '12/20/2013'
from datetime import datetime

# Define a function to convert the date column to datetime
def convert_date(date_str):
    return datetime.strptime(date_str, '%m/%d/%Y').date()

date_format_rdd = sales_rdd.map(lambda x: (x[0],x[1],x[2],x[3],x[4],convert_date(x[5]), ...))
date_format_sorted_rdd = date_format_rdd.sortBy(lambda x : x[5], ascending=False)
date_format_sorted_rdd.take(10)

Out[26]: [('Asia',
  'Bhutan',
  'Cereal',
  'Offline',
  'M',
  datetime.date(2017, 7, 28),
  Ellipsis),
 ('Sub-Saharan Africa',
  'Senegal',
  'Cosmetics',
  'Online',
  'C',
  datetime.date(2017, 7, 26),
  Ellipsis),
 ('Middle East and North Africa',
  'United Arab Emirates',
  'Household',
  'Online',
  'C',
  datetime.date(2017, 7, 26),
  Ellipsis),
 ('Australia and Oceania',
  'Australia',
  'Beverages',
  'Online',
  'L',
  datetime.date(2017, 7, 26),
  Ellipsis),
 ('Sub-Saharan Africa',
  "Cote d'Ivoire",
  'Vegetables',
  'Online',
  'H',
  datetime.date(2017, 7, 24),
  Ellipsis),
 ('Sub-Saharan Africa',
  'Chad',
  'Household',
  'Online',
  'L',
  datetime.date(2017, 7, 24),
  Ellipsis),
 ('Australia and Oceania',
  'Vanuatu',
  'Office Supplies',
  'Online',
  'C',
  datetime.date(2017, 7, 24),
  Ellipsis),
 ('Europe',
  'Kosovo',
  'Vegetables',
  'Offline',
  'C',
  datetime.date(2017, 7, 23),
  Ellipsis),
 ('Europe',
  'San Marino',
  'Snacks',
  'Offline',
  'C',
  date

In [0]:
date_format_sorted_rdd.coalesce(1).saveAsTextFile('/FileStore/tables/recent_sales')

In [0]:
# 4.Display the products with atleast 2 occurences of 'a' (Using spark)
def product_count(product):
   cnt = 0
   for ch in product:
      if ch == 'a':
         cnt = cnt + 1
   if cnt > 1:
      return True
   return False

product_rdd = sales_rdd.filter(lambda x : product_count(x[2]))
product_rdd.count()
product_rdd.collect()

Out[29]: [['Middle East and North Africa',
  'Morocco',
  'Personal Care',
  'Offline',
  'L',
  '11/8/2010',
  '412882792',
  '11/22/2010',
  '48',
  '81.73',
  '56.67',
  '3923.04',
  '2720.16',
  '1202.88'],
 ['Sub-Saharan Africa',
  'Benin',
  'Personal Care',
  'Online',
  'L',
  '8/7/2016',
  '440603101',
  '9/15/2016',
  '3104',
  '81.73',
  '56.67',
  '253689.92',
  '175903.68',
  '77786.24'],
 ['Middle East and North Africa',
  'Saudi Arabia',
  'Personal Care',
  'Offline',
  'L',
  '2/2/2014',
  '688270556',
  '3/4/2014',
  '3407',
  '81.73',
  '56.67',
  '278454.11',
  '193074.69',
  '85379.42'],
 ['Sub-Saharan Africa',
  'Central African Republic',
  'Personal Care',
  'Offline',
  'H',
  '12/16/2014',
  '863838946',
  '1/5/2015',
  '7995',
  '81.73',
  '56.67',
  '653431.35',
  '453076.65',
  '200354.70'],
 ['Australia and Oceania',
  'Tuvalu',
  'Personal Care',
  'Offline',
  'M',
  '12/21/2016',
  '963251912',
  '2/1/2017',
  '6501',
  '81.73',
  '56.67',
  '531326.73'

In [0]:
product_rdd.coalesce(1).saveAsTextFile('/FileStore/tables/products_withaa')

In [0]:
# 5.Display country in each region with highest units sold. (Using spark)

country_region_units_rdd = sales_rdd.map(lambda x : ((x[0],x[1]),int(x[8])))
country_region_units_rdd.take(10)
country_region_units_rdd = country_region_units_rdd.reduceByKey(lambda a,b : a+b)
country_region_units_rdd.take(10)
country_region_units_rdd.map(lambda x: (x[0][0],(x[0][1],x[1]))).reduceByKey(lambda a,b : max(a,b ,key=lambda x : x[1])).collect()
region_rdd = country_region_units_rdd.map(lambda x : ((x[0][0]),(x[0][1],x[1]))) 
region_rdd.take(10)
region_rdd = region_rdd.reduceByKey(lambda a,b : max(a,b ,key=lambda x : x[1]))
region_rdd.collect()

Out[33]: [('Asia', ('Myanmar', 199967)),
 ('Australia and Oceania', ('Australia', 183909)),
 ('Middle East and North Africa', ('Somalia', 193065)),
 ('Central America and the Caribbean', ('Grenada', 205943)),
 ('Europe', ('Macedonia', 203078)),
 ('Sub-Saharan Africa', ('Equatorial Guinea', 197767)),
 ('North America', ('United States of America', 159519))]

In [0]:
region_rdd.coalesce(1).saveAsTextFile('/FileStore/tables/region_country_with_maxunitssold')

In [0]:
# 6.Display the unit price and unit cost of each item in ascending order. (Using spark)

item_rdd = sales_rdd.map(lambda x : (x[2],round(float(x[9]),4),round(float(x[10]),4)))
item_rdd = item_rdd.distinct().sortBy(lambda x : x[0])
item_rdd.collect()


Out[36]: [('Baby Food', 255.28, 159.42),
 ('Beverages', 47.45, 31.79),
 ('Cereal', 205.7, 117.11),
 ('Clothes', 109.28, 35.84),
 ('Cosmetics', 437.2, 263.33),
 ('Fruits', 9.33, 6.92),
 ('Household', 668.27, 502.54),
 ('Meat', 421.89, 364.69),
 ('Office Supplies', 651.21, 524.96),
 ('Personal Care', 81.73, 56.67),
 ('Snacks', 152.58, 97.44),
 ('Vegetables', 154.06, 90.93)]

In [0]:
item_rdd.coalesce(1).saveAsTextFile('/FileStore/tables/item_unitprice_unitcost')

In [0]:
# 7.Display the number of sales yearwise. (Using pyspark)
sales_year_rdd = sales_rdd.map(lambda x : (int(x[5].split('/')[2]), 1))
sales_year_rdd = sales_year_rdd.reduceByKey(lambda a,b : a + b).sortBy(lambda x : x[0])
sales_year_rdd.collect()

Out[38]: [(2010, 632),
 (2011, 658),
 (2012, 678),
 (2013, 660),
 (2014, 660),
 (2015, 679),
 (2016, 670),
 (2017, 363)]

In [0]:
sales_year_rdd.coalesce(1).saveAsTextFile('/FileStore/tables/sales_yearwise')

In [0]:
# 8.Display the number of orders for each item. (Using pyspark)
items_rdd = sales_rdd.map(lambda x : (x[2],1))
items_rdd = items_rdd.reduceByKey(lambda a,b : a+b)
items_rdd.collect()


Out[40]: [('Baby Food', 445),
 ('Snacks', 398),
 ('Cereal', 385),
 ('Clothes', 386),
 ('Cosmetics', 424),
 ('Fruits', 447),
 ('Beverages', 447),
 ('Personal Care', 415),
 ('Office Supplies', 420),
 ('Meat', 399),
 ('Vegetables', 410),
 ('Household', 424)]

In [0]:
items_rdd.coalesce(1).saveAsTextFile('/FileStore/tables/item_ordercount')